In [2]:
import os
path = os.getcwd()

if path[1:5] == "Home":
    %run /Home/siv30/wad005/master/GRB/src/read_matlab_file.ipynb
    %run /Home/siv30/wad005/master/GRB/src/convert_erg.ipynb
    %run /Home/siv30/wad005/master/GRB/src/safety_time.ipynb
else:
    %run /Users/andreas/phys/master/GRB/src/read_matlab_file.ipynb
    %run /Users/andreas/phys/master/GRB/src/convert_erg.ipynb
    %run /Users/andreas/phys/master/GRB/src/safety_time.ipynb

All MAC paths are set!
read_matlab_file.ipynb loaded successfully!
All MAC paths are set!
read_matlab_file.ipynb loaded successfully!
convert_erg.ipynb loaded successfully!
All MAC paths are set!
read_matlab_file.ipynb loaded successfully!
All MAC paths are set!
read_matlab_file.ipynb loaded successfully!
convert_erg.ipynb loaded successfully!
safety_time.ipynb loaded successfully!


# imports for MAC
%run /Users/andreas/phys/master/GRB/src/read_matlab_file.ipynb
%run /Users/andreas/phys/master/GRB/src/convert_erg.ipynb
%run /Users/andreas/phys/master/GRB/src/safety_time.ipynb

# implementation blueprint
        
    GENERAL IMPLEMENTATION STEPS
    ----------------------------    
    
    
    DATA HANDLING
    -------------
    1. Subtract absolute time correction from T0 and tus array
    2. importing calibration coef and coverting to keV
    3. creating dataframe containing data
    4. sorting by adr and then tus
    5. estimating dt between i and i-1 (after sorting by adr and tus)
    6. calculating safety time based on keV and dt
    7. flagging events that have dt <= st AND typ > 1 events.

    LC PRODUCTION
    -----------------
    1. creating LC containing complete photon list. DONE
    2. filtering photon list based on relevant energy interval. HED [350-1500 kev], LED [80-350 kev]. DONE
    3. remove flagged events from the filtered photon list
    4. calculate background for the relevant energy bands 
    5. plotting LC where background is subtracted
    
    
    final step. plot LC of the flagged events only. The ratio between the flagged events / number of counts in each bin is a measure of the dead time. (deadtime relative to binsize) 
    
    
    

# Notes
    --final plots should be done in the current notebook

    - be careful of using is True or is not True. There are two types of booleans: np.bool_ or native python bool
    USE: == or !=
    
    - a lot of energies are converted to keV = 351.4519739
    
    - Flagging all type 3 events or using the algorithm to find it? 116 vs 114
    
    -flagg all type 3 events manually?

# complete df

In [1]:
def complete_df(grb_id=None, remove_flag=False, lag=None, flag_cut="HED"):
    '''
    TODO: Use np.float32 on cal_abc array
    
    
    
    input: grb_id <str>: Name of gammaray burst id of type GRB200415
    remove_flag <bool>: standard to False. If the returned df will contain any flagged events
    lag <float> [ms] : lag is added to the tus array (corrolated with another spacecraft). 
    
    notes: Valleys or typ=3 events will be removed from the df anyway
    Long GRB's have to be concatenated
    
    If flag=True the returned df will not contain any st flagged events
       
    else: keeping flagged events
    
    return: <pd.dataframe> df sorted by tus in increasing order
    '''
    
    
    if len(trig_dict_HED[grb_id])>1:
        print("Creating df for long GRB")
        
        #handling the long GRBs
        
        #Need dau & det arrays for the energy calibration
        # ---> Making them flow through with tus and erg.
        
        #need also typ and adr array
        tus_cut, erg_cut, typ_cut,adr_cut, dau_cut, det_cut, T0 = cut_cat(grb_id=grb_id, flag=flag_cut) #cutcat gets the concatinated arrays from grb_id
        
        keV, cal_abc = convert_erg(erg_cut,dau_cut,det_cut,T0)
        print("erg converted to keV")
        
        df = create_df(tus_cut, erg_cut, typ_cut, adr_cut, dau_cut, det_cut, keV, cal_abc)
        # same handling of the df from here 
        #----------------------------------------------------
       
    
    else:
        print("Creating df for short GRB")
        tus, erg, typ, adr, dau, det, T0 = read_matlab_file_with_BGO_data(trig_dict_HED[grb_id][0])
        keV,cal_abc = convert_erg(erg, dau, det, T0)
        print("erg converted to keV")
        df = create_df(tus, erg, typ, adr, dau,det ,keV, cal_abc)
        
        # same handling of the df from here 
        #----------------------------------------------------
        
    df = df.sort_values(by=['adr','tus']) #important to sort first by the adr then tus --> Handling the triggers in each detector seperate
    print("df sorted by adr and tus")
    #Creating columns for st, dt and flag
    st_list = get_st(df)
    df["st"] = st_list

    dt_list = get_dt(df)
    df["dt"] = dt_list

    flag_arr = get_flag(df) #should also flag fast events!
    df["flag"] = flag_arr

    print("safety time events flagged. " + "Found " + str(sum(df["flag"].to_numpy())) + " flags")

    if remove_flag==False:
        print("returning df containing flagged events")
        return df,T0
    else:
        flag_label = []
        for flag_idx,row in df.iterrows():
            if row["flag"]==True:
                flag_label.append(flag_idx)

        if len(flag_label) == 0:
            return print("No flags found. Check for error")

        else:
            print("Removed " + str(len(flag_label)) + " flags from df")
            df.drop(labels=flag_label,inplace=True)
            print("df sorted by tus")
            df.sort_values(by=['tus'],inplace=True)
            
            if lag != None:
                df["tus"] = df["tus"].to_numpy() + lag*1e3 #converting lag in ms into us
                return df, T0
            else:
                return df, T0

TODO: vectorize the flag column and do this to remove the flagged events...

flag = df_GRB210619["flag"].to_numpy()
for i in np.arange(0,len(flag)):

    if flag[i] == np.bool_(True):
        #remove flag index
       

# filtering energy interval and removing flagged events

In [ ]:
# df is soreted by adr and then tus

def filter_by_energy_interval(df,lower_bound,upper_bound):
    """lower and upper bound given in keV
    
    Note: have to remove the flagged ST events because one can not trust the energy for those flagged events

    return: tus: photon trigger list filtered by energy interval
            kev: array of keV's 
            flagged: array containing the flaged tus in the energy interval"""
    
    tus_arr = df["tus"].to_numpy()
    kev_arr = df["keV"].to_numpy()
    flag_arr = df["flag"].to_numpy()
    
    
    tus_temp, kev_temp, flagged_temp = [],[],[]
    print("selecting kev interval. Removing flagged events.")
    for i,kev in enumerate(kev_arr):
        if lower_bound <= kev <= upper_bound:
            if flag_arr[i] == False:
                kev_temp.append(kev)
                tus_temp.append(tus_arr[i])
            else:
                flagged_temp.append(tus_arr[i])

    print("found {} out of {} flagged triggers on the given keV interval".format(len(flagged_temp), sum(df["flag"])))
    return np.array(tus_temp), np.array(kev_temp), np.array(flagged_temp)
    

# background estimation

In [ ]:
def estimate_background(tus,start=None,end=None,binsize=None):
    """Estimates background for a given tus array on a given ms interval.
    Using histograms for estimation. 
    -Constant fit: sum(bin counts)/length(bins)
    -Binsize given in ms. USING BINS ESTIMATED FROM TUS FOR ALL OTHER LC'S!
    -indexing of the bins hist[start_idx, end_idx) >> up to but not including the end_idx bin
    """
 
    
    if binsize == None:
        return print("binsize need to be specified")
    
    hist = np.histogram(tus/1e3,bins=np.arange(
            min(tus/1e3), max(tus/1e3) + binsize, binsize)) #binning in ms not us
    
    hist_bg = hist[0]
    edges = hist[1] #array of position the hist bin start
    
    if start=="start" or start==None:
        start_idx = 0
    else:
        start_idx = find_nearest(edges,start)
        
    if end == None:
        return print("end index need to be specified")
    else:
        end_idx = find_nearest(edges,end)
        bg = round(sum(hist_bg[start_idx:end_idx])/len(hist_bg[start_idx:end_idx]))
        print("bg avg on interval:", bg)
        #bg = 220
        hist_no_bg = hist_bg - bg #subtracting background all bins.
        
    return hist_bg, hist_no_bg, edges, bg

In [ ]:
def estimate_dead_time(df=None,binsize=None):
    '''defined as the the ratio between the counts in the filtered (or the normal array) / removed bins
    generating the ratio between the normal and filtered hists
    
    
    NOTE: binning based on the TUS_FILT array.
    
    estimate the ratio between the number of flagged events vs filtered counts in each bin. 
    how to estimate dead time? 
    
    return the ratio pr bin as an array'''
    
    tus = df["tus"].to_numpy()
    flag = df["flag"].to_numpy()
    
    mask_noflag, mask_flag = np.where(flag == False)[0], np.where(flag == True)[0]

    tus_noflag = tus[mask_noflag]
    tus_flag = tus[mask_flag]
    
    hist_noflag, edges_noflag = np.histogram(tus_noflag/1e3,bins=np.arange(min(tus)/1e3,max(tus)/1e3 + binsize, binsize))
    hist_flag, edges_flag = np.histogram(tus_flag/1e3,bins=np.arange(min(tus)/1e3,max(tus)/1e3 + binsize, binsize))
    
    fraq = []
    
    for i,bin_count in enumerate(hist_noflag):
        if bin_count == 0:
            fraq.append(0)
        else:
            fraq.append(hist_flag[i]/bin_count)
    
    return fraq,edges_noflag
    

df_test,T0 = complete_df(grb_id="GRB200415")
fraq,edges_noflag = estimate_dead_time(df=df_test,binsize=2)
plt.bar(x=edges_noflag[:-1], height=fraq,label= "max fraq: {} \n avg: {}".format(round(max(fraq),5),round(sum(fraq)/len(fraq),3)))
plt.xlabel("T-T0 [2 ms bins]")
plt.ylabel("fraction")
plt.title("GRB200415, valleys removed")
plt.legend()                                                                                
plt.show()

# main filter 

In [ ]:
#change name to main()
def main_filter(grb_id=None, lower_bound=None, upper_bound=None, start=None, end=None, binsize=None):
    '''Need plots for original image, filtered energy range with background marked, filtered with background removed
    
    returns: filtered histogram, edges and background

    '''
    #data handling
    df,T0 = complete_df(grb_id)
    
    #regular tus array
    tus = df["tus"].to_numpy()
    org_hist, org_edges = np.histogram(tus/1e3,bins=np.arange(min(tus/1e3), max(tus/1e3) + binsize, binsize))
    
    #filtered arrays
    tus_filt, kev_filt, tus_flagged = filter_by_energy_interval(df,lower_bound,upper_bound)
    
    #histogram binning
    hist_bg, hist_no_bg, edges, bg = estimate_background(tus_filt,start=start,end=end,binsize=binsize)
    flagged_hist, flagged_edges = np.histogram(tus_flagged/1e3,bins=np.arange(min(tus/1e3), max(tus/1e3) + binsize, binsize))
    
    #deadtime 
    #dead_time = estimate_dead_time(tus_filt_hist=hist_no_bg,tus_flagged_hist=flagged_hist)
    
    return org_hist, hist_bg, hist_no_bg, flagged_hist, org_edges, bg, binsize
    

In [ ]:
#org_hist, hist, hist_no_bg, flagged_hist, org_edges, bg, binsize = main_filter(grb_id="GRB200415",lower_bound=350,upper_bound=1500,start=-750,end=-20,binsize=2)

# plotting functions

In [ ]:
def LC_plots(org_hist, hist, hist_no_bg, flagged_hist, edges, bg, binsize,export=False):
     # Producing LC
    #to silence an axis for odd number of plots: ax[-1, -1].axis('off')
    fig, (ax1,ax2,ax3) = plt.subplots(4,1,sharey=True,figsize=(16,15))
    

    ax1.bar(x=edges[:-1], height=org_hist, width=binsize, edgecolor="black",align='edge',color="black",label="Full photon list")
    
    ax2.bar(x=edges[:-1], height=hist, width=binsize, edgecolor= "black",align='edge', color="black", label="Energy range {}-{} keV".format(lower_bound,upper_bound))

    ax2.axhline(bg,c="r",label="Background: {} counts/bin \n Background interval [{},{}] ms".format(bg,start,end))
    
    ax3.bar(edges[:-1], hist_no_bg ,width=binsize, edgecolor="black", align='edge',color="black", label="Background subtracted. \n Energy range: {}-{} keV".format(lower_bound,upper_bound))

    #ax4.bar(x=edges[:-1], height=flagged_hist, width=binsize, edgecolor="black", align='edge',color="black", label="flagged events")
    

    #setting X/Y labels and legends
    
    ax1.set_xlabel("T-T0 [ms], binsize = {} ms".format((binsize)))
    ax1.set_ylabel("Counts/ {} ms".format((binsize)))
    ax1.legend()
    
    ax2.set_xlabel("T-T0 [ms], binsize = {} ms".format((binsize)))
    ax2.set_ylabel("Counts/ {} ms".format((binsize)))
    ax2.legend()

    ax3.set_xlabel("T-T0 [ms], binsize = {} ms".format((binsize)))
    ax3.set_ylabel("Counts/ {} ms".format((binsize)))
    ax3.legend()
    
    
    #setting titles for figure and subfigures
    date_id = datetime.strftime(T0,format='%y%m%d') #Correct format for title of plot: GRBYYMMDD
    time_id = datetime.strftime(T0,format='%H:%M:%S.%f')[0:12]
    grb_id_title = "GRB " + date_id
    plt.suptitle(grb_id_title + " , " + "T0 = " + time_id,fontweight='bold')
    ax1.set_title("Complete photon array")
    ax2.set_title("Filtered by energy interval with background, flagged events removed")
    ax3.set_title("Filtered by energy interval without background, flagged events removed")
    #ax2[1].set_title("Flagged events on energy interval")
    
    if export is False:
        plt.tight_layout()
        plt.show()
    else:
        plt.tight_layout()
        d = "/Home/siv30/wad005/master/GRB/pyplots/LC/"
        plt.savefig(fname=d+grb_id_title)
        print("PDF generated")
        #crop_pdfs(d)

# TODO: 

    -plot the fraction of flagged (exclude valleys)/norm+fast events. Use scatter plot and hist plot


    lower_bound=350
    upper_bound=1500
    start=-450
    end=450
    binsize=2
    org_hist, hist, hist_no_bg, flagged_hist, org_edges, bg, binsize = main_filter(grb_id="GRB190206",lower_bound=lower_bound,upper_bound=upper_bound,start=start,end=end,binsize=binsize)


## Fraq of flagged events

In [ ]:
#df,T0 = complete_df(grb_id="GRB200415")
#print(df.shape)
#df.head()

In [ ]:
#flag_label=[]
#for label,row in df.iterrows():
#    if row["flag"] == False:
#        flag_label.append(label)
#        
#df_flag = df.drop(labels=flag_label)
#print(df_flag.shape)
#df_flag.head()

tus_flagged = df_flag["tus"].to_numpy()
tus_org = df["tus"].to_numpy()

bins=np.arange(min(tus_org)/1e3,max(tus_org)/1e3+2,2)

hist_flag ,edges_flag = np.histogram(tus_flagged/1e3,bins=bins)
hist_org,edges_org = np.histogram(tus_org/1e3,bins=bins)

hist_fraq = []

for i,count in enumerate(hist_org):
    if count == 0:
        hist_fraq.append(0)
    else:
        fraq = hist_flag[i]/count
        hist_fraq.append(fraq)

fig,(ax1) = plt.subplots(1,3,figsize=(20,9))


ax1[0].bar(x=edges_org[:-1], height=hist_org, width=2,align="edge")
ax1[1].bar(x=edges_flag[:-1], height=hist_flag, width=2,align="edge")
ax1[2].bar(x=edges_org[:-1],height=hist_fraq, width=2,align="edge")

ax1[0].set_title("LC with valleys removed")
ax1[0].set_xlabel("T-T0 [2ms bins]")
ax1[0].set_ylabel("counts / 2ms bin")

ax1[1].set_title("Flagged events")
ax1[1].set_xlabel("T-T0 [2ms bins]")
ax1[1].set_ylabel("counts / 2ms bin")


ax1[2].set_title("Fraction of flagged vs all bin counts")
ax1[2].set_xlabel("T-T0 [2ms bins]")
ax1[2].set_ylabel("fraction")
plt.tight_layout()
plt.show()

In [ ]:
def export_df_flag(org_hist=None,hist=None,hist_no_bg=None,edges=None,bg=None,binsize=None):
    data = {"Full photon list bin count":org_hist, "Filtered (by=energy flags) bin count":hist, 
            "Filtered+ (by=energy flags background) bin count. bg is {} counts pr {} ms bin".format(bg,binsize): hist_no_bg,
           "left bin edge [ms]": np.round(edges[:-1],6)}
    df = pd.DataFrame(data)
    return df
    
#df_exp = export_df_flag(org_hist=org_hist,hist=hist,hist_no_bg=hist_no_bg,edges=org_edges,bg=bg,binsize=binsize)

In [ ]:
print("LC_calibration loaded successfully!")